# Imports

In [147]:
import json
import requests

import numpy as np
import pandas as pd

# Load data

In [148]:
df = pd.read_csv("data/data.csv").drop("Unnamed: 0", axis=1)

In [149]:
TARGET="SeriousDlqin2yrs"

## Test API

### define API URLs

In [150]:
lambda_api_url = "https://x82fudbwt4.execute-api.eu-west-1.amazonaws.com/dev/predict"
sagemaker_api_url = "https://4m53ma6yf8.execute-api.eu-central-1.amazonaws.com/test/credit-explainer"
local_api_url = "http://localhost:5001/predict"

api = local_api_url

### Send request

In [152]:
sample_df = df.sample(1).drop(TARGET, axis=1)
sample_json = sample_df.to_json(orient='records')
header = {'Content-Type': 'application/json', 'Accept': 'application/json'}

resp = requests.post(api, data=json.dumps(sample_json), headers=header)

resp.json()

{'prediction': [0.021695483500824356],
 'shap_base': 0.06013555555555556,
 'shap_values': {'MonthlyIncome': [0.00047997136926613323],
  'age': [0.002401008774263085],
  'NumberOfDependents': [-0.0010044244852816695],
  'DebtRatio': [-0.0012618673829351973],
  'RevolvingUtilizationOfUnsecuredLines': [-0.01750849094491076],
  'NumberRealEstateLoansOrLines': [-0.0001247615889879619],
  'NumberOfOpenCreditLinesAndLoans': [-0.0005640194333750395],
  'NumberOfTime30-59DaysPastDueNotWorse': [-0.006885185238433925],
  'NumberOfTime60-89DaysPastDueNotWorse': [-0.0041087349609328595],
  'NumberOfTimes90DaysLate': [-0.009863568163403006]}}

### Format response

In [153]:
preds = np.asarray(resp.json()['prediction'])
shap_base = resp.json()['shap_base']
shap_values = pd.DataFrame(resp.json()['shap_values'])[sample_df.columns].values[0]

In [154]:
contrib_df = get_contrib_df(shap_base, shap_values, sample_df.iloc[[0]])
contrib_df

,col,contribution,value,cumulative,base
0,_BASE,0.060136,,0.060136,0.000000
1,RevolvingUtilizationOfUnsecuredLines,-0.017508,0,0.042627,0.060136
2,NumberOfTimes90DaysLate,-0.009864,0,0.032763,0.042627
3,NumberOfTime30-59DaysPastDueNotWorse,-0.006885,0,0.025878,0.032763
4,NumberOfTime60-89DaysPastDueNotWorse,-0.004109,0,0.021770,0.025878
5,age,0.002401,54,0.024171,0.021770
6,DebtRatio,-0.001262,0.161512,0.022909,0.024171
7,NumberOfDependents,-0.001004,0,0.021904,0.022909
8,NumberOfOpenCreditLinesAndLoans,-0.000564,8,0.021340,0.021904
9,MonthlyIncome,0.000480,2036,0.021820,0.021340
